In [1]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *

/home/azureuser/Ambarish/sentence-llama-index/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(URL,port=6333)

In [3]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [4]:
from llama_index.llms.azure_openai import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [6]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",max_length=512
)

In [7]:
from llama_index.core import Settings
from llama_index.core  import VectorStoreIndex

Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [9]:
query_engine = index.as_query_engine()

In [10]:
#query = "What is  ENERGY INTENSITY AT TSJ?"
#query = "Describe about Tata Steel Kalinganagar?"
query = "What should the government do about road safety?"

In [11]:
response = query_engine.query(query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
print(response.get_formatted_sources())
print("query was:", query)
print("answer was:", response)

> Source (Doc id: 68bda1d6-1ec4-4916-8918-b93aa5f14d7d): /circle6Develop a multidisciplinary approach toroad safety.


> Source (Doc id: 2bff9cf0-b5f0-48f9-8941-103b87bcb169): The World Health
Organization also declared the year 2004 as the
Year of Road Safety and launched...
query was: What should the government do about road safety?
answer was: A multidisciplinary approach should be developed for road safety, according to the provided context information. There is no specific mention of what the government should do about road safety.


In [13]:
from pprint import pprint

In [14]:
pprint(str(response))

('A multidisciplinary approach should be developed for road safety, according '
 'to the provided context information. There is no specific mention of what '
 'the government should do about road safety.')


In [15]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

In [16]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    llm = llm,
)

In [17]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

In [18]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(query)

In [19]:
pprint(str(response))

('Road safety is a shared responsibility of the government and a range of '
 'civil society stakeholders. Therefore, the government should develop a '
 'multidisciplinary approach to road safety in order to effectively and '
 'sustainably prevent road traffic injuries, which are a major global public '
 'health problem according to the World Report on Road Traffic Injury '
 'Prevention.')
